### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 4


### Дисциплина: Управление ИТ-сервисами и контентом

##### Студент:  Матюхин Григорий Васильевич
##### Группа:   НПИбд-01-21

## Москва 2024
***

# Введение

В данной лабораторной работе необходимо для какого-либо метода из лабораторной №3 составить JSON схему, описывающую все возможности метода. JSON схема должна быть составлена и для объекта, который посылается в запросе и для объекта, который приходит в ответе от сервера. Правильность схемы проверьте с помощью валидатора, например https://jsonschemavalidator.net

# Выполнение

Для данной лабораторной работы мной был выбран метод `photos.createAlbum`. Этот метод хорош тем, что его ответы имеют вложенные схемы и мы можем это протестировать.

Сохраним наши схемы и тесты для них в файлы. Тестировать будем с помощью утилиты `check-jsonschema`.

## Запрос

In [1]:
cat ./schema/photos.createAlbum.json | jq

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "type": "object",
  "properties": {
    "v": {
      "type": "string",
      "description": "Версия используемого API",
      "const": "5.199"
    },
    "access_token": {
      "type": "string",
      "description": "Токен доступа (должен быть пользовательский)"
    },
    "title": {
      "description": "Album title.",
      "type": "string",
      "minLength": 1
    },
    "group_id": {
      "description": "ID of the community in which the album will be created.",
      "type": "integer",
      "format": "int64",
      "entity": "owner"
    },
    "description": {
      "description": "Album description.",
      "type": "string"
    },
    "privacy_view": {
      "type": "array",
      "default": "all",
      "items": {
        "type": "string"
      }
    },
    "privacy_comment": {
      "type": "array",
      "default": "all",
      "items": {
        "type": "string"
      }
    },
    "upload_by_admins_only": {
 

### Тесты

Минимальный запрос

In [2]:
cat ./tests/minimal_req.json | jq

{
  "v": "5.199",
  "access_token": "supersecret",
  "title": "My album title"
}


In [3]:
check-jsonschema ./tests/minimal_req.json \
--schemafile ./schema/photos.createAlbum.json \
--base-uri "$(pwd)/schema/"

ok -- validation done


Минимальный плохой запрос

In [4]:
cat ./tests/bad_req.json | jq

{
  "v": "5.199",
  "title": "My album title"
}


In [5]:
check-jsonschema ./tests/bad_req.json \
--schemafile ./schema/photos.createAlbum.json \
--base-uri "$(pwd)/schema/"

Schema validation errors were encountered.
  ./tests/bad_req.json::$: 'access_token' is a required property


: 1

Плохой тип ключа

In [6]:
cat ./tests/bad_req_type.json | jq

{
  "v": 5,
  "access_token": "supersecret",
  "title": "My album title"
}


In [7]:
check-jsonschema ./tests/bad_req_type.json \
--schemafile ./schema/photos.createAlbum.json \
--base-uri "$(pwd)/schema/"

Schema validation errors were encountered.
  ./tests/bad_req_type.json::$.v: 5 is not of type 'string'
  ./tests/bad_req_type.json::$.v: '5.199' was expected


: 1

Большой запрос

In [8]:
cat ./tests/complex_req.json | jq

{
  "v": "5.199",
  "access_token": "supersecret",
  "title": "My album title",
  "group_id": 2123,
  "description": "Photos of me",
  "privacy_view": [
    "me",
    "you"
  ]
}


In [9]:
check-jsonschema ./tests/complex_req.json \
--schemafile ./schema/photos.createAlbum.json \
--base-uri "$(pwd)/schema/"

ok -- validation done


## Ответ

Ответ состоит из нескольких вложенных частей. На самом деле `full_album` можно вложить в `response`, но для болшей общности (так как все данные и для других методов подаются в ключе `response`) я решил оставить его отдельно.

In [10]:
cat ./schema/response.json | jq

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "type": "object",
  "properties": {
    "response": {
      "$ref": "full_album.json"
    }
  },
  "required": [
    "response"
  ]
}


In [11]:
cat ./schema/full_album.json | jq

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "type": "object",
  "properties": {
    "can_upload": {
      "description": "Information whether current user can upload photo to the album",
      "type": "boolean"
    },
    "comments_disabled": {
      "description": "Information whether album comments are disabled",
      "type": "boolean"
    },
    "created": {
      "type": "integer",
      "description": "Date when the album has been created in Unixtime, not set for system albums",
      "minimum": 0
    },
    "description": {
      "type": "string",
      "description": "Photo album description"
    },
    "can_delete": {
      "type": "boolean",
      "description": "album can delete"
    },
    "id": {
      "type": "integer",
      "description": "Photo album ID"
    },
    "can_include_to_feed": {
      "type": "boolean",
      "description": "album can be selected to feed"
    },
    "is_locked": {
      "type": "boolean",
      "description": "Need show 

In [12]:
cat ./schema/photo_sizes.json | jq

{
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "type": "object",
  "properties": {
    "height": {
      "type": "integer",
      "description": "Height in px",
      "minimum": 0
    },
    "url": {
      "type": "string",
      "format": "uri",
      "description": "URL of the image"
    },
    "src": {
      "type": "string",
      "format": "uri",
      "description": "URL of the image"
    },
    "type": {
      "type": "string",
      "description": "Size type",
      "enum": [
        "t",
        "s",
        "m",
        "x",
        "o",
        "p",
        "q",
        "r",
        "k",
        "l",
        "y",
        "z",
        "c",
        "w",
        "a",
        "b",
        "e",
        "i",
        "d",
        "j",
        "temp",
        "h",
        "g",
        "n",
        "f",
        "max",
        "base",
        "u",
        "v",
        "orig"
      ]
    },
    "width": {
      "type": "integer",
      "description": "Width in px",
  

### Тесты

Минимальный ответ

In [13]:
cat ./tests/minimal_resp.json | jq

{
  "response": {
    "title": "My album title",
    "id": 12345,
    "owner_id": 123456,
    "size": 1
  }
}


In [14]:
check-jsonschema ./tests/minimal_resp.json \
--schemafile ./schema/response.json \
--base-uri "$(pwd)/schema/"

ok -- validation done


Минимальный плохой ответ

In [15]:
cat ./tests/bad_resp.json | jq

{
  "response": {
    "id": 12345,
    "owner_id": 123456,
    "size": 1
  }
}


In [16]:
check-jsonschema ./tests/bad_resp.json \
--schemafile ./schema/response.json \
--base-uri "$(pwd)/schema/"

Schema validation errors were encountered.
  ./tests/bad_resp.json::$.response: 'title' is a required property


: 1

Сложный запрос

In [17]:
cat ./tests/complex_resp.json | jq

{
  "response": {
    "title": "My album title",
    "id": 12345,
    "owner_id": 123456,
    "size": 1,
    "can_upload": true,
    "created": 0,
    "can_include_to_feed": false
  }
}


In [18]:
check-jsonschema ./tests/complex_resp.json \
--schemafile ./schema/response.json \
--base-uri "$(pwd)/schema/"

ok -- validation done


Еще более сложный запрос

In [19]:
cat ./tests/complex_complex_resp.json | jq

{
  "response": {
    "title": "My album title",
    "id": 12345,
    "owner_id": 123456,
    "size": 1,
    "can_upload": true,
    "created": 0,
    "can_include_to_feed": false,
    "sizes": [
      {
        "height": 512,
        "width": 512,
        "type": "v"
      }
    ]
  }
}


Ошибка в сложном запросе

In [20]:
check-jsonschema ./tests/complex_complex_resp.json \
--schemafile ./schema/response.json \
--base-uri "$(pwd)/schema/"

ok -- validation done


In [21]:
cat ./tests/bad_complex_complex_resp.json | jq

{
  "response": {
    "title": "My album title",
    "id": 12345,
    "owner_id": 123456,
    "size": 1,
    "can_upload": true,
    "created": 0,
    "can_include_to_feed": false,
    "sizes": [
      {
        "height": 512,
        "width": 512,
        "type": "who"
      }
    ]
  }
}


In [22]:
check-jsonschema ./tests/bad_complex_complex_resp.json \
--schemafile ./schema/response.json \
--base-uri "$(pwd)/schema/"

Schema validation errors were encountered.
  ./tests/bad_complex_complex_resp.json::$.response.sizes[0].type: 'who' is not one of ['t', 's', 'm', 'x', 'o', 'p', 'q', 'r', 'k', 'l', 'y', 'z', 'c', 'w', 'a', 'b', 'e', 'i', 'd', 'j', 'temp', 'h', 'g', 'n', 'f', 'max', 'base', 'u', 'v', 'orig']


: 1